In [1]:
import re
from collections import defaultdict

import numpy as np
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt

In [2]:
with open('../../data/clean/misconduct_indicators_train.pkl', 'rb') as train_file:
    misconduct = pickle.load(train_file)

In [3]:
dummyized_cols = [
    'misconduct_type',
    'court_type',
    'enforcement_agency',
    'contracting_party'
]

indicator_features = [
    col for col in misconduct.columns
    if re.search(f'^({"|".join(dummyized_cols)})_', col)
]

In [4]:
def cv_mean(neighbors, p_norm):
    return np.mean(
        cross_val_score(
            KNeighborsClassifier(p=p_norm, weights='distance', n_neighbors=neighbors),
            misconduct.loc[:, indicator_features],
            misconduct['disposition_type'],
            cv=3 # limited by class imbalance
        )
    )

# accuracy_by_k_by_p = defaultdict(dict)

# for p in [1, 2, 3, 4]:
#     for k in range(1, 60):
#         accuracy_by_k_by_p[p][k] = cv_mean(k, p)

# plt.plot(
#     list(accuracy_by_k_by_p[1].keys()),
#     list(accuracy_by_k_by_p[1].values()),
#     color='#b02060'
# )

In [5]:
# accuracy peaked at ~10 and ~30
# ~30 barely lower and less variance in region
cv_mean(p_norm=1, neighbors=30)

0.6041951268607311